In [1]:
import re
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import glob
import os

## Concatenate all the files

In [3]:
df = pd.read_csv("/content/drive/MyDrive/DATA SCIENCE/Projects/Laptop price prediction/DS project/laptop_data.csv")

In [4]:
df.head()

,Unnamed: 0,name,cpu,ram,monitor,gpu,memory,battery,connection,weight,price
0,0,0,0,0,0,0,0,10WHr,0,0,0
1,748,Lenovo ThinkPad E15 Gen 4,0,0,0,0,0,10WHr,0,0,0
2,749,Lenovo IdeaPad Gaming 3i Gen 7,0,0,0,0,0,10WHr,0,0,0
3,750,Lenovo ThinkPad P15v Gen 3,0,0,0,0,0,10WHr,0,0,0
4,751,Lenovo Legion S7 Gen 7,0,0,0,0,0,10WHr,0,0,0


In [5]:
df.shape

(646, 11)

### Remove duplicates

In [6]:
df = df.drop_duplicates(ignore_index = True)
del df[df.columns[0]]

In [7]:
df.shape

(646, 10)

In [8]:
df.dtypes

name          object
cpu           object
ram           object
monitor       object
gpu           object
memory        object
battery       object
connection    object
weight        object
price          int64
dtype: object

In [9]:
df["battery"] = df["battery"].replace("10WHr", "0")

### Transform *price*, *battery*, *weight* to the right data type

In [10]:
df["battery"]

0            0
1            0
2            0
3            0
4            0
        ...   
641      80WHr
642    99.9WHr
643      80WHr
644      82WHr
645      99WHr
Name: battery, Length: 646, dtype: object

In [11]:
df["battery"] = df["battery"].str.replace("WHr", "")

In [12]:
df["battery"] = df["battery"].astype("float")

In [13]:
df["weight"] = df["weight"].str.replace("kg", "")

In [14]:
df["weight"] = df["weight"].astype("float")

### Dropping the rows with missing values in the *price* columns and the rows with more than 5 features

In [15]:
df = df.replace(["0", 0], np.nan)

In [16]:
df = df.dropna(subset = ['price'], how = "any")
df = df.dropna(thresh = 5)
df = df.reset_index(drop=True)
df

,name,cpu,ram,monitor,gpu,memory,battery,connection,weight,price
0,Sạc Macbook Hyperjuice Ultimate Charger 110W D...,"Intel Core i5 1235U, 10 nhân, 12 luồng","8GB, DDR4, 3200 MHz","14"", 1920 x 1080 px, IPS, 65% sRGB, 60 Hz",Intel Iris Xe Graphics,SSD 512GB,39.3,"1 x Type-C, 1 x USB-A",1.400,3590000.0
1,Ghế Công Thái Học HyperWork HW01 (Arm 3D - Gray),"Intel Core i5 11300H, 4 nhân, 8 luồng","16GB, LPDDR4X, 4266 MHz","14.4"", 2400 x 1600 px, PixelSense Flow Display...",Intel Iris Xe Graphics,SSD 512GB,58.0,2 x Type-C,1.743,3790000.0
2,Màn hình chuyên đồ hoạ ViewSonic VG2755-2K (QH...,"AMD Ryzen R5-5625U, 6 nhân, 12 luồng","8GB, DDR4, 3200 MHz","15.6"", 1920 x 1080 px, IPS, Chống chói, 65% sRGB",AMD Radeon Graphics,SSD 512GB,39.3,"1 x Type-C, 1 x USB-A",1.700,7790000.0
3,Asus Vivobook X515 (Chính Hãng) (X515MA-BR481T),"Intel Celeron N4020, 2 nhân, 2 luồng","4GB, DDR4, 3200 MHz","15.6"", 1366 x 768 px, TN, Matte, 60 Hz",Intel UHD Graphics,"SSD 256GB, HDD 1024GB",37.0,"1 x Type-C, 3 x USB-A",NaN,7990000.0
4,ASUS Vivobook X515JA,"Intel Core i3 1005G1, 2 nhân, 4 luồng","8GB, 3200 MHz","15.6"", 1366 x 768 px, TN, Chống chói, 45% sRGB",Intel UHD Graphics,SSD 256GB,37.0,"1 x Type-C, 2 x USB-A",1.800,8990000.0
...,...,...,...,...,...,...,...,...,...,...
555,Razer Blade 15 Advanced (Intel Gen 11) (RZ09-0...,"Intel Core i9 11900H, 8 nhân, 16 luồng","32GB, DDR4, 3200 MHz","15.6"", 3840 x 2160 px, OLED, Glossy, Cảm ứng, ...",Intel UHD,"SSD 1024GB, HDD 0GB",80.0,"2 x Type-C, 3 x USB-A, Thunderbolt",2.010,114990000.0
556,MSI Stealth GS77 2022,"Intel Core i9-12900H, 16 nhân, 24 luồng","32GB, DDR5, 4800 MHz","17.3"", 3840x2160 px, IPS, Chống chói, 120 Hz",Nvidia Geforce RTX 3080Ti 16GB GDDR6,SSD 2TB,99.9,"2 x Type-C, 2 x USB-A, Thunderbolt",2.800,119990000.0
557,Razer Blade 15 2022,"Intel Core i9 12900H, 14 nhân, 20 luồng","32GB, DDR5, 4800 MHz","15.6"", 3840 x 2160 px, IPS, 100% sRGB, 144 Hz","Intel® Iris® Xe Graphics eligible, RTX30080Ti",SSD 1024GB,80.0,"1 x Type-C, 2 x USB-A",2.010,129990000.0
558,Razer Blade 17 2022,"Intel Core i9 12900H, 14 nhân, 20 luồng","32GB, 4800 MHz","17.3"", 3840 x 2160 px, IPS, 100% sRGB, 144 Hz","Intel Iris Xe Graphics, Nvidia Geforce RTX3080Ti",SSD 1024GB,82.0,"2 x Type-C, 3 x USB-A, Thunderbolt",2.750,129990000.0


### Check the number of missing values

In [17]:
df.isnull().sum()

name           0
cpu            0
ram            1
monitor        0
gpu            3
memory         0
battery       15
connection     0
weight        55
price          0
dtype: int64

### Add new feature: the laptop brand

In [18]:
brand_dict = {"Lenovo": ["Lenovo", "ThinkPad", "Thinkpad"],
              "Apple": ["Apple"],
              "HP": ["HP", "Hp"],
              "Dell": ["Dell", "New Inspiron", "Alienware"],
              "Asus": ["Asus", "ASUS"],
              "Acer": ["Acer"],
              "Microsoft":["Microsoft", "Surface"],
              "Huawei": ["Huawei", "HUAWEI"],
              "Xiaomi": ["Xiaomi"],
              "LG": ["LG"],
              "GIGABYTE": ["GIGABYTE"],
              "Samsung": ["Samsung"],
              "Razer": ["Razer"],
              "MSI": ["MSI"],
              "AVITA": ["AVITA"],
              "Colorful": ["Colorful"]}

In [19]:
def get_brand(brand_name, brand_dict = brand_dict):
  for i in brand_dict.keys():
    for j in brand_dict[i]:
      if j in brand_name:
        return i
  return "Other"

In [20]:
df["brand"] = df["name"].apply(get_brand)

In [21]:
df.head()

,name,cpu,ram,monitor,gpu,memory,battery,connection,weight,price,brand
0,Sạc Macbook Hyperjuice Ultimate Charger 110W D...,"Intel Core i5 1235U, 10 nhân, 12 luồng","8GB, DDR4, 3200 MHz","14"", 1920 x 1080 px, IPS, 65% sRGB, 60 Hz",Intel Iris Xe Graphics,SSD 512GB,39.3,"1 x Type-C, 1 x USB-A",1.400,3590000.0,Other
1,Ghế Công Thái Học HyperWork HW01 (Arm 3D - Gray),"Intel Core i5 11300H, 4 nhân, 8 luồng","16GB, LPDDR4X, 4266 MHz","14.4"", 2400 x 1600 px, PixelSense Flow Display...",Intel Iris Xe Graphics,SSD 512GB,58.0,2 x Type-C,1.743,3790000.0,Other
2,Màn hình chuyên đồ hoạ ViewSonic VG2755-2K (QH...,"AMD Ryzen R5-5625U, 6 nhân, 12 luồng","8GB, DDR4, 3200 MHz","15.6"", 1920 x 1080 px, IPS, Chống chói, 65% sRGB",AMD Radeon Graphics,SSD 512GB,39.3,"1 x Type-C, 1 x USB-A",1.700,7790000.0,Other
3,Asus Vivobook X515 (Chính Hãng) (X515MA-BR481T),"Intel Celeron N4020, 2 nhân, 2 luồng","4GB, DDR4, 3200 MHz","15.6"", 1366 x 768 px, TN, Matte, 60 Hz",Intel UHD Graphics,"SSD 256GB, HDD 1024GB",37.0,"1 x Type-C, 3 x USB-A",NaN,7990000.0,Asus
4,ASUS Vivobook X515JA,"Intel Core i3 1005G1, 2 nhân, 4 luồng","8GB, 3200 MHz","15.6"", 1366 x 768 px, TN, Chống chói, 45% sRGB",Intel UHD Graphics,SSD 256GB,37.0,"1 x Type-C, 2 x USB-A",1.800,8990000.0,Asus


### Split the monitor column into its *size, resolution, panel, display* and *technology used*

In [22]:
df['monitor'].value_counts()[100:150]

14", 1366 x 768 px, TN, Matte, 60 Hz                                 2
15.6", 1920 x 1080 px, IPS, Chống chói, 60 Hz                        2
14", 1920 x 1080 px, IPS, Matte, Cảm ứng, 95% sRGB, 60 Hz            2
14", 1920x1080 px, IPS, Chống chói                                   2
14", 1920x1080 px, IPS, LED Backlit, Chống chói Anti glare, 60 Hz    2
15.6", 3840 x 2400 px, IPS, Gương, Cảm ứng, 100% sRGB, 60 Hz         1
17.3", 1920 x 1080 px, IPS, Chống chói, 100% sRGB, 144 Hz            1
15.6", 1920×1080 px, IPS, Chống chói, 100% sRGB, 144 Hz              1
13", 2160 x 1350 px, IPS, Matte, Cảm ứng, 100% sRGB, 60 Hz           1
15", 1920 x 1080 px, IPS, Matte, , 94% sRGB, 144 Hz                  1
15", 3840 x 2160 px, IGZO, Matte, , 96% sRGB, 60 Hz                  1
13", 1920 x 1080 px, IPS, Matte, 97% sRGB, 60 Hz                     1
15.6", 1920 x 1080 px, IPS, Matte, Cảm ứng, 95% sRGB, 60 Hz          1
15.6", 1920 x 1080 px, IPS, Chống chói, 62..5% sRGB, 144 Hz          1
14", 2

In [23]:
monitor = df["monitor"].str.split(", ").to_frame()

In [24]:
monitor

,monitor
0,"[14"", 1920 x 1080 px, IPS, 65% sRGB, 60 Hz]"
1,"[14.4"", 2400 x 1600 px, PixelSense Flow Displa..."
2,"[15.6"", 1920 x 1080 px, IPS, Chống chói, 65% s..."
3,"[15.6"", 1366 x 768 px, TN, Matte, 60 Hz]"
4,"[15.6"", 1366 x 768 px, TN, Chống chói, 45% sRGB]"
...,...
555,"[15.6"", 3840 x 2160 px, OLED, Glossy, Cảm ứng,..."
556,"[17.3"", 3840x2160 px, IPS, Chống chói, 120 Hz]"
557,"[15.6"", 3840 x 2160 px, IPS, 100% sRGB, 144 Hz]"
558,"[17.3"", 3840 x 2160 px, IPS, 100% sRGB, 144 Hz]"


In [25]:
def size(x):
    try:
        for i in x:
            if "\"" in i:
                return i.replace("\"", "")
    except:
        return x

def refresh_rate(x):
    try:
        for i in x:
            if "Hz" in i and len(i) <= 8:
                return i.replace(" Hz", "")
    except:
        return x

def resolution_x(x):
    try:
        for i in x:
            if i == "1080 px":
              return "1920"
            if bool(re.search(r'.+x.+|.+x.+px|.+X.+|.+X.+px', i)):
                reso_split = re.split(r"X|x", i)
                return reso_split[0].replace(".", "")
    except:
        return x

def resolution_y(x):
    try:
        for i in x:
            if i == "1080 px":
              return "1080"
            if bool(re.search(r'.+x.+|.+x.+px|.+X.+|.+X.+px', i)):
                reso_split = re.split(r"X|x", i)
                return reso_split[1].strip(" p").replace(".", "")
    except:
        return x

def panel(x):
    try:
        for i in x:
            if bool(re.match(r"WVA",i)):
                return "WVA"
            if bool(re.match(r"IPS",i)):
                return "IPS"
            if bool(re.match(r"TN",i)):
                return "TN"
            if bool(re.match(r"Retina",i)):
                return "Retina"
            if bool(re.match(r"OLED",i)):
                return "OLED"
            if bool(re.match(r"QLED",i)):
                return "QLED"
            if bool(re.match(r"AMOLED",i)):
                return "AMOLED"
            if bool(re.match(r"IGZO",i)):
                return "IGZO"
            if bool(re.match(r"TFT",i)):
                return "TFT"
    except:
        return x

def display(x):
    try:
        for i in x:
            if i == "Glossy":
                return i
            if i == "Matte":
                return i
    except:
        return x

def blue_light_filter(x):
    try: 
        for i in x:
            if i == "Giảm ánh sáng xanh":
                return 1
        return 0
    except:
        return x

def touch_screen(x):
    try: 
        for i in x:
            if i == "Cảm ứng":
                return 1
        return 0
    except:
        return x

def LED_backlit(x):
    try: 
        for i in x:
            if "backlit" in i.lower():
                return 1
        return 0
    except:
        return x

def anti_glare(x):
    try: 
        for i in x:
            if "Chống chói" in i:
                return 1
        return 0
    except:
        return x

def sRGB(x):
    try:
        for i in x:
            if "sRGB" in i:
              sRGB_num = re.sub(r"% sRGB|%% sRGB|", "",i)
              return sRGB_num.replace(",", ".").replace("..", ".")
    except:
        return x

Replace the monitor feature by 11 different features:
- monitor_size: 
- monitor_resolution_x
- monito_resolution_y
- monitor_panel
- monitor_display
- monitor_touch_screen
- monitor_blue_light_filter
- monitor_LED_backlit
- monitor_anti_glare
- monitor_sRGB
- monitor_refresh_rate

In [26]:
df["monitor_size"] = monitor["monitor"].apply(size)
df["monitor_resolution_x"] = monitor["monitor"].apply(resolution_x)
df["monitor_resolution_y"] = monitor["monitor"].apply(resolution_y)
df["monitor_panel"] = monitor["monitor"].apply(panel)
df["monitor_display"] = monitor["monitor"].apply(display)
df["monitor_torch_screen"] = monitor["monitor"].apply(touch_screen)
df["monitor_blue_light_filter"] = monitor["monitor"].apply(blue_light_filter)
df["monitor_LED_backlit"] = monitor["monitor"].apply(LED_backlit)
df["monitor_anti_glare"] = monitor["monitor"].apply(anti_glare)
df["monitor_sRGB"] = monitor["monitor"].apply(sRGB)
df["monitor_refresh_rate"] = monitor["monitor"].apply(refresh_rate)

In [27]:
df["monitor_panel"].value_counts(dropna = False)

IPS       401
WVA        46
OLED       46
TN         28
NaN        18
Retina     13
TFT         3
IGZO        2
AMOLED      2
QLED        1
Name: monitor_panel, dtype: int64

In [28]:
df.tail(2)

,name,cpu,ram,monitor,gpu,memory,battery,connection,weight,price,...,monitor_resolution_x,monitor_resolution_y,monitor_panel,monitor_display,monitor_torch_screen,monitor_blue_light_filter,monitor_LED_backlit,monitor_anti_glare,monitor_sRGB,monitor_refresh_rate
558,Razer Blade 17 2022,"Intel Core i9 12900H, 14 nhân, 20 luồng","32GB, 4800 MHz","17.3"", 3840 x 2160 px, IPS, 100% sRGB, 144 Hz","Intel Iris Xe Graphics, Nvidia Geforce RTX3080Ti",SSD 1024GB,82.0,"2 x Type-C, 3 x USB-A, Thunderbolt",2.75,129990000.0,...,3840,2160,IPS,None,0,0,0,0,100,144
559,MSI Titan GT77 2022 (12UHS),"Intel Core i9-12900HX Processor, 16 nhân, 24 l...","64GB, DDR5, 4800 MHz","17.3"", 3840x2160 px, IPS, Chống chói, 120 Hz",NVIDIA GeForce RTX 3080 Ti Laptop GPU 16GB GDDR6,SSD 2TB,99.0,"2 x Type-C, 3 x USB-A, Thunderbolt",3.30,139990000.0,...,3840,2160,IPS,None,0,0,0,1,None,120


### Replace the connection feature by 3 different features:
- connection_type_c: 
- connection_usb_a
- connection_thunderbolt

In [29]:
df["connection"] = df["connection"].str.strip(",")

In [30]:
df["connection"].value_counts()

1 x Type-C, 2 x USB-A                 112
1 x Type-C, 3 x USB-A                  76
2 x Type-C, 2 x USB-A, Thunderbolt     63
1 x Type-C, 2 x USB-A, Thunderbolt     37
2 x Type-C, 2 x USB-A                  34
2 x Type-C, Thunderbolt                32
2 x Type-C, 1 x USB-A, Thunderbolt     30
1 x Type-C, 3 x USB-A, Thunderbolt     29
3 x USB-A                              19
2 x Type-C, 3 x USB-A, Thunderbolt     18
1 x Type-C, 1 x USB-A                  16
2 x Type-C, 1 x USB-A                  16
3 x Type-C, Thunderbolt                10
2 x Type-C, 4 x USB-A                   7
1 x Type-C, 1 x USB-A, Thunderbolt      7
3 x Type-C                              6
2 x USB-A                               6
1 x Type-C, 4 x USB-A                   5
1 x Type-C, Thunderbolt                 5
1 x Type-C                              4
3 x Type-C, 3 x USB-A                   4
3 x Type-C, 1 x USB-A                   2
2 x Type-C, 0 x USB-A, Thunderbolt      2
4 x Type-C, Thunderbolt           

In [31]:
connection = df["connection"].str.split(", ").to_frame()
connection

,connection
0,"[1 x Type-C, 1 x USB-A]"
1,[2 x Type-C]
2,"[1 x Type-C, 1 x USB-A]"
3,"[1 x Type-C, 3 x USB-A]"
4,"[1 x Type-C, 2 x USB-A]"
...,...
555,"[2 x Type-C, 3 x USB-A, Thunderbolt]"
556,"[2 x Type-C, 2 x USB-A, Thunderbolt]"
557,"[1 x Type-C, 2 x USB-A]"
558,"[2 x Type-C, 3 x USB-A, Thunderbolt]"


In [32]:
def type_c(x):
    try: 
        for i in x:
            if "Type-C" in i:
                return i.replace(" x Type-C", "")
        return 0
    except:
        return x

def usb_a(x):
    try: 
        for i in x:
            if "USB-A" in i:
                return i.replace(" x USB-A", "")
        return 0
    except:
        return x
    
def thunderbolt(x):
    try: 
        for i in x:
            if "Thunderbolt" in i:
                return 1
        return 0
    except:
        return x

In [33]:
df["connection_type_c"] = connection["connection"].apply(type_c)
df["connection_usb_a"] = connection["connection"].apply(usb_a)
df["connection_thunderbolt"] = connection["connection"].apply(thunderbolt)

In [34]:
df.head()

,name,cpu,ram,monitor,gpu,memory,battery,connection,weight,price,...,monitor_display,monitor_torch_screen,monitor_blue_light_filter,monitor_LED_backlit,monitor_anti_glare,monitor_sRGB,monitor_refresh_rate,connection_type_c,connection_usb_a,connection_thunderbolt
0,Sạc Macbook Hyperjuice Ultimate Charger 110W D...,"Intel Core i5 1235U, 10 nhân, 12 luồng","8GB, DDR4, 3200 MHz","14"", 1920 x 1080 px, IPS, 65% sRGB, 60 Hz",Intel Iris Xe Graphics,SSD 512GB,39.3,"1 x Type-C, 1 x USB-A",1.400,3590000.0,...,None,0,0,0,0,65,60,1,1,0
1,Ghế Công Thái Học HyperWork HW01 (Arm 3D - Gray),"Intel Core i5 11300H, 4 nhân, 8 luồng","16GB, LPDDR4X, 4266 MHz","14.4"", 2400 x 1600 px, PixelSense Flow Display...",Intel Iris Xe Graphics,SSD 512GB,58.0,2 x Type-C,1.743,3790000.0,...,None,1,0,0,0,100,120,2,0,0
2,Màn hình chuyên đồ hoạ ViewSonic VG2755-2K (QH...,"AMD Ryzen R5-5625U, 6 nhân, 12 luồng","8GB, DDR4, 3200 MHz","15.6"", 1920 x 1080 px, IPS, Chống chói, 65% sRGB",AMD Radeon Graphics,SSD 512GB,39.3,"1 x Type-C, 1 x USB-A",1.700,7790000.0,...,None,0,0,0,1,65,None,1,1,0
3,Asus Vivobook X515 (Chính Hãng) (X515MA-BR481T),"Intel Celeron N4020, 2 nhân, 2 luồng","4GB, DDR4, 3200 MHz","15.6"", 1366 x 768 px, TN, Matte, 60 Hz",Intel UHD Graphics,"SSD 256GB, HDD 1024GB",37.0,"1 x Type-C, 3 x USB-A",NaN,7990000.0,...,Matte,0,0,0,0,None,60,1,3,0
4,ASUS Vivobook X515JA,"Intel Core i3 1005G1, 2 nhân, 4 luồng","8GB, 3200 MHz","15.6"", 1366 x 768 px, TN, Chống chói, 45% sRGB",Intel UHD Graphics,SSD 256GB,37.0,"1 x Type-C, 2 x USB-A",1.800,8990000.0,...,None,0,0,0,1,45,None,1,2,0


### Replace the ram feature by 3 different features:
- ram_capacity: 
- ram_type
- ram_speed

In [35]:
df["ram"].value_counts()[50:100]

12GB, 3200 MHz                      1
, 3200 MHz                          1
4GB, LPDDR4X, 4266 MHz              1
64GB, DDR4, 2666 MHz                1
16GB, LPDDR4X, 2666 MHz             1
32GB, DDR4, 2389 MHz                1
32GB, 3200 MHz                      1
8GB, 4267 MHz                       1
8GB, LPDDDR5                        1
16GB, LPDDR4X, 0 MHz                1
16GB, LPDDR5                        1
16GB, LPDDR4x                       1
32GB                                1
8GB, 5200 MHz                       1
32GB, LPDDR5, 4800 MHz              1
64GB, 4800 MHz                      1
4GB, 2400 MHz                       1
32GB, DDR4, 2400 MHz                1
8GB, LPDDR3, 1866 MHz               1
4GB                                 1
16GB, LPDDR5, 5800 MHz              1
32GB, DDR4, 2666 MHz                1
4GB, DdR4, 3200 MHz                 1
8GB, 4266 MHz                       1
8GB, LPDDR4x                        1
16GB, 3800 MHz                      1
16GB, LPDDR3

In [36]:
ram = df["ram"].str.split(", ").to_frame()
ram.iloc[2,0]

['8GB', 'DDR4', '3200 MHz']

In [37]:
def capacity(x):
    try: 
        for i in x:
            if "GB" in i:
                return i.replace("GB", "")
        return np.nan
    except:
        return x

def ram_type(x):
    try: 
        for i in x:
            if i == "DDR4":
                return i
            if "DDR5" in i:
                return i
            if i == "LPDDR5":
                return i
            if i == "LPDDR4x":
                return i
            if i == "LPDDR3L":
                return i
            if i == "LPDDR3":
                return i
            if i == "DDR3L":
                return i
        return np.nan
    except:
        return x
    
def speed(x):
    try: 
        for i in x:
            if "MHz" in i:
                return i.replace(" MHz", "")
        return np.nan
    except:
        return x

In [38]:
df["ram_capacity"] = ram["ram"].apply(capacity)
df["ram_type"] = ram["ram"].apply(ram_type)
df["ram_speed"] = ram["ram"].apply(speed)

In [39]:
df["ram_speed"].value_counts(dropna=False)

3200    250
4266     61
4800     53
NaN      41
2666     32
2400     29
5200     27
2933     25
2133     15
3733      9
4267      6
6400      4
1600      1
1866      1
5800      1
3800      1
2665      1
4277      1
2389      1
0         1
Name: ram_speed, dtype: int64

In [40]:
df.head()

,name,cpu,ram,monitor,gpu,memory,battery,connection,weight,price,...,monitor_LED_backlit,monitor_anti_glare,monitor_sRGB,monitor_refresh_rate,connection_type_c,connection_usb_a,connection_thunderbolt,ram_capacity,ram_type,ram_speed
0,Sạc Macbook Hyperjuice Ultimate Charger 110W D...,"Intel Core i5 1235U, 10 nhân, 12 luồng","8GB, DDR4, 3200 MHz","14"", 1920 x 1080 px, IPS, 65% sRGB, 60 Hz",Intel Iris Xe Graphics,SSD 512GB,39.3,"1 x Type-C, 1 x USB-A",1.400,3590000.0,...,0,0,65,60,1,1,0,8,DDR4,3200
1,Ghế Công Thái Học HyperWork HW01 (Arm 3D - Gray),"Intel Core i5 11300H, 4 nhân, 8 luồng","16GB, LPDDR4X, 4266 MHz","14.4"", 2400 x 1600 px, PixelSense Flow Display...",Intel Iris Xe Graphics,SSD 512GB,58.0,2 x Type-C,1.743,3790000.0,...,0,0,100,120,2,0,0,16,NaN,4266
2,Màn hình chuyên đồ hoạ ViewSonic VG2755-2K (QH...,"AMD Ryzen R5-5625U, 6 nhân, 12 luồng","8GB, DDR4, 3200 MHz","15.6"", 1920 x 1080 px, IPS, Chống chói, 65% sRGB",AMD Radeon Graphics,SSD 512GB,39.3,"1 x Type-C, 1 x USB-A",1.700,7790000.0,...,0,1,65,None,1,1,0,8,DDR4,3200
3,Asus Vivobook X515 (Chính Hãng) (X515MA-BR481T),"Intel Celeron N4020, 2 nhân, 2 luồng","4GB, DDR4, 3200 MHz","15.6"", 1366 x 768 px, TN, Matte, 60 Hz",Intel UHD Graphics,"SSD 256GB, HDD 1024GB",37.0,"1 x Type-C, 3 x USB-A",NaN,7990000.0,...,0,0,None,60,1,3,0,4,DDR4,3200
4,ASUS Vivobook X515JA,"Intel Core i3 1005G1, 2 nhân, 4 luồng","8GB, 3200 MHz","15.6"", 1366 x 768 px, TN, Chống chói, 45% sRGB",Intel UHD Graphics,SSD 256GB,37.0,"1 x Type-C, 2 x USB-A",1.800,8990000.0,...,0,1,45,None,1,2,0,8,NaN,3200


### Replace the memory feature by 2 different features:
- HDD_capacity: 
- SSD_capacity:

In [41]:
df["memory"].value_counts()

SSD 512GB                268
SSD 256GB                109
SSD 512GB, HDD 0GB        57
SSD 1024GB                49
SSD 256GB, HDD 0GB        23
SSD 1024GB, HDD 0GB       18
SSD 128GB                 10
SSD 2TB                    9
SSD 512GB, HDD 1GB         6
SSD 2TB, HDD 0GB           3
SSD 1000GB, HDD 0GB        2
SSD 1.001953125TB          1
SSD 4TB                    1
HDD 1024GB                 1
SSD 256GB, HDD 2TB         1
SSD 256GB, HDD 1024GB      1
SSD 1.9765625TB            1
Name: memory, dtype: int64

In [42]:
df["memory"] = df["memory"].str.replace("HDD 0GB", "").str.strip(" , ")
df["memory"] = df["memory"].str.replace("1TB", "1024GB")
df["memory"] = df["memory"].str.replace("2TB", "2048GB")
df["memory"] = df["memory"].str.replace("1.9765625TB", "2048GB")
df["memory"] = df["memory"].str.replace("1.001953125TB", "1024GB")
df["memory"] = df["memory"].str.replace("4TB", "4096GB")

df.loc[df["memory"] == "","memory"] = np.nan

<ipython-input-42-5636d94ae61a>:4: FutureWarning: The default value of regex will change from True to False in a future version.
  df["memory"] = df["memory"].str.replace("1.9765625TB", "2048GB")
<ipython-input-42-5636d94ae61a>:5: FutureWarning: The default value of regex will change from True to False in a future version.
  df["memory"] = df["memory"].str.replace("1.001953125TB", "1024GB")


In [43]:
df["memory"].value_counts()

SSD 512GB                325
SSD 256GB                132
SSD 1024GB                68
SSD 2048GB                13
SSD 128GB                 10
SSD 512GB, HDD 1GB         6
SSD 1000GB                 2
SSD 256GB, HDD 1024GB      1
HDD 1024GB                 1
SSD 256GB, HDD 2048GB      1
SSD 4096GB                 1
Name: memory, dtype: int64

In [44]:
memory= df["memory"].str.split(",").to_frame()
memory

,memory
0,[SSD 512GB]
1,[SSD 512GB]
2,[SSD 512GB]
3,"[SSD 256GB, HDD 1024GB]"
4,[SSD 256GB]
...,...
555,[SSD 1024GB]
556,[SSD 2048GB]
557,[SSD 1024GB]
558,[SSD 1024GB]


In [45]:
def hdd_capacity(x):
    try:
        for i in x:
          if "HDD" in i:
             return re.sub(r"\D", "", i)
        return 0
    except:
        return x

def ssd_capacity(x):
    try:
        for i in x:
          if "SSD" in i:
             return re.sub(r"\D", "", i)
        return 0
    except:
        return x

In [46]:
df["HDD_capacity"] = memory["memory"].apply(hdd_capacity)
df["SSD_capacity"] = memory["memory"].apply(ssd_capacity)

In [47]:
df.tail()

,name,cpu,ram,monitor,gpu,memory,battery,connection,weight,price,...,monitor_sRGB,monitor_refresh_rate,connection_type_c,connection_usb_a,connection_thunderbolt,ram_capacity,ram_type,ram_speed,HDD_capacity,SSD_capacity
555,Razer Blade 15 Advanced (Intel Gen 11) (RZ09-0...,"Intel Core i9 11900H, 8 nhân, 16 luồng","32GB, DDR4, 3200 MHz","15.6"", 3840 x 2160 px, OLED, Glossy, Cảm ứng, ...",Intel UHD,SSD 1024GB,80.0,"2 x Type-C, 3 x USB-A, Thunderbolt",2.01,114990000.0,...,None,60,2,3,1,32,DDR4,3200,0,1024
556,MSI Stealth GS77 2022,"Intel Core i9-12900H, 16 nhân, 24 luồng","32GB, DDR5, 4800 MHz","17.3"", 3840x2160 px, IPS, Chống chói, 120 Hz",Nvidia Geforce RTX 3080Ti 16GB GDDR6,SSD 2048GB,99.9,"2 x Type-C, 2 x USB-A, Thunderbolt",2.80,119990000.0,...,None,120,2,2,1,32,DDR5,4800,0,2048
557,Razer Blade 15 2022,"Intel Core i9 12900H, 14 nhân, 20 luồng","32GB, DDR5, 4800 MHz","15.6"", 3840 x 2160 px, IPS, 100% sRGB, 144 Hz","Intel® Iris® Xe Graphics eligible, RTX30080Ti",SSD 1024GB,80.0,"1 x Type-C, 2 x USB-A",2.01,129990000.0,...,100,144,1,2,0,32,DDR5,4800,0,1024
558,Razer Blade 17 2022,"Intel Core i9 12900H, 14 nhân, 20 luồng","32GB, 4800 MHz","17.3"", 3840 x 2160 px, IPS, 100% sRGB, 144 Hz","Intel Iris Xe Graphics, Nvidia Geforce RTX3080Ti",SSD 1024GB,82.0,"2 x Type-C, 3 x USB-A, Thunderbolt",2.75,129990000.0,...,100,144,2,3,1,32,NaN,4800,0,1024
559,MSI Titan GT77 2022 (12UHS),"Intel Core i9-12900HX Processor, 16 nhân, 24 l...","64GB, DDR5, 4800 MHz","17.3"", 3840x2160 px, IPS, Chống chói, 120 Hz",NVIDIA GeForce RTX 3080 Ti Laptop GPU 16GB GDDR6,SSD 2048GB,99.0,"2 x Type-C, 3 x USB-A, Thunderbolt",3.30,139990000.0,...,None,120,2,3,1,64,DDR5,4800,0,2048


### Replace the cpu feature by 3 different features:
- cpu_name: 
- cpu_core_num
- cpu_thread_num

In [48]:
with pd.option_context('display.max_rows', None,):
   print(df["cpu"].value_counts())
# df["cpu"].value_counts()[150:200]

Intel Core i5 1135G7, 4 nhân, 8 luồng                           38
Intel Core i7 11800H, 8 nhân, 16 luồng                          23
Intel Core i7 1165G7, 4 nhân, 8 luồng                           22
Intel Core i5-1135G7, 4 nhân, 8 luồng                           17
AMD Ryzen 5 5600H, 6 nhân, 12 luồng                             17
AMD Ryzen 7 5800H, 8 nhân, 16 luồng                             14
Intel Core i7-11800H, 8 nhân, 16 luồng                          12
Intel Core i7-12700H, 14 nhân, 20 luồng                         12
Intel Core i5-1240P, 12 nhân, 16 luồng                          11
AMD Ryzen 5 5500U, 6 nhân, 12 luồng                             11
AMD Ryzen 7 5700U, 8 nhân, 16 luồng                             10
Intel Core i7-1260P, 12 nhân, 16 luồng                          10
Intel Core i7 10750H, 6 nhân, 12 luồng                           9
Intel Core i7 8665U, 4 nhân, 8 luồng                             8
Intel Core i5 1235U, 10 nhân, 12 luồng                        

In [49]:
cpu_brand_dict = {"Intel":["Intel","Core"],
                  "AMD": ["Ryzen", "AMD"],
                  "Apple":["Apple"]}

In [50]:
def get_cpu_brand(brand_name, cpu_brand_dict = cpu_brand_dict):
  for i in cpu_brand_dict.keys():
    for j in cpu_brand_dict[i]:
      if j in brand_name:
        return i
  return "Other"

In [51]:
df["cpu_brand"] = df["cpu"].apply(get_cpu_brand)

In [52]:
cpu = df["cpu"].str.split(", ")
cpu.head()

0    [Intel Core i5 1235U, 10 nhân, 12 luồng]
1     [Intel Core i5 11300H, 4 nhân, 8 luồng]
2      [AMD Ryzen R5-5625U, 6 nhân, 12 luồng]
3      [Intel Celeron N4020, 2 nhân, 2 luồng]
4     [Intel Core i3 1005G1, 2 nhân, 4 luồng]
Name: cpu, dtype: object

In [53]:
df[["cpu_name", "cpu_core_num", "cpu_thread_num"]] = cpu.apply(pd.Series).rename(columns = {0:"cpu_name", 1: "cpu_core_num", 2: "cpu_thread_num"})
df

,name,cpu,ram,monitor,gpu,memory,battery,connection,weight,price,...,connection_thunderbolt,ram_capacity,ram_type,ram_speed,HDD_capacity,SSD_capacity,cpu_brand,cpu_name,cpu_core_num,cpu_thread_num
0,Sạc Macbook Hyperjuice Ultimate Charger 110W D...,"Intel Core i5 1235U, 10 nhân, 12 luồng","8GB, DDR4, 3200 MHz","14"", 1920 x 1080 px, IPS, 65% sRGB, 60 Hz",Intel Iris Xe Graphics,SSD 512GB,39.3,"1 x Type-C, 1 x USB-A",1.400,3590000.0,...,0,8,DDR4,3200,0,512,Intel,Intel Core i5 1235U,10 nhân,12 luồng
1,Ghế Công Thái Học HyperWork HW01 (Arm 3D - Gray),"Intel Core i5 11300H, 4 nhân, 8 luồng","16GB, LPDDR4X, 4266 MHz","14.4"", 2400 x 1600 px, PixelSense Flow Display...",Intel Iris Xe Graphics,SSD 512GB,58.0,2 x Type-C,1.743,3790000.0,...,0,16,NaN,4266,0,512,Intel,Intel Core i5 11300H,4 nhân,8 luồng
2,Màn hình chuyên đồ hoạ ViewSonic VG2755-2K (QH...,"AMD Ryzen R5-5625U, 6 nhân, 12 luồng","8GB, DDR4, 3200 MHz","15.6"", 1920 x 1080 px, IPS, Chống chói, 65% sRGB",AMD Radeon Graphics,SSD 512GB,39.3,"1 x Type-C, 1 x USB-A",1.700,7790000.0,...,0,8,DDR4,3200,0,512,AMD,AMD Ryzen R5-5625U,6 nhân,12 luồng
3,Asus Vivobook X515 (Chính Hãng) (X515MA-BR481T),"Intel Celeron N4020, 2 nhân, 2 luồng","4GB, DDR4, 3200 MHz","15.6"", 1366 x 768 px, TN, Matte, 60 Hz",Intel UHD Graphics,"SSD 256GB, HDD 1024GB",37.0,"1 x Type-C, 3 x USB-A",NaN,7990000.0,...,0,4,DDR4,3200,1024,256,Intel,Intel Celeron N4020,2 nhân,2 luồng
4,ASUS Vivobook X515JA,"Intel Core i3 1005G1, 2 nhân, 4 luồng","8GB, 3200 MHz","15.6"", 1366 x 768 px, TN, Chống chói, 45% sRGB",Intel UHD Graphics,SSD 256GB,37.0,"1 x Type-C, 2 x USB-A",1.800,8990000.0,...,0,8,NaN,3200,0,256,Intel,Intel Core i3 1005G1,2 nhân,4 luồng
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
555,Razer Blade 15 Advanced (Intel Gen 11) (RZ09-0...,"Intel Core i9 11900H, 8 nhân, 16 luồng","32GB, DDR4, 3200 MHz","15.6"", 3840 x 2160 px, OLED, Glossy, Cảm ứng, ...",Intel UHD,SSD 1024GB,80.0,"2 x Type-C, 3 x USB-A, Thunderbolt",2.010,114990000.0,...,1,32,DDR4,3200,0,1024,Intel,Intel Core i9 11900H,8 nhân,16 luồng
556,MSI Stealth GS77 2022,"Intel Core i9-12900H, 16 nhân, 24 luồng","32GB, DDR5, 4800 MHz","17.3"", 3840x2160 px, IPS, Chống chói, 120 Hz",Nvidia Geforce RTX 3080Ti 16GB GDDR6,SSD 2048GB,99.9,"2 x Type-C, 2 x USB-A, Thunderbolt",2.800,119990000.0,...,1,32,DDR5,4800,0,2048,Intel,Intel Core i9-12900H,16 nhân,24 luồng
557,Razer Blade 15 2022,"Intel Core i9 12900H, 14 nhân, 20 luồng","32GB, DDR5, 4800 MHz","15.6"", 3840 x 2160 px, IPS, 100% sRGB, 144 Hz","Intel® Iris® Xe Graphics eligible, RTX30080Ti",SSD 1024GB,80.0,"1 x Type-C, 2 x USB-A",2.010,129990000.0,...,0,32,DDR5,4800,0,1024,Intel,Intel Core i9 12900H,14 nhân,20 luồng
558,Razer Blade 17 2022,"Intel Core i9 12900H, 14 nhân, 20 luồng","32GB, 4800 MHz","17.3"", 3840 x 2160 px, IPS, 100% sRGB, 144 Hz","Intel Iris Xe Graphics, Nvidia Geforce RTX3080Ti",SSD 1024GB,82.0,"2 x Type-C, 3 x USB-A, Thunderbolt",2.750,129990000.0,...,1,32,NaN,4800,0,1024,Intel,Intel Core i9 12900H,14 nhân,20 luồng


In [54]:
df["cpu_core_num"] = df["cpu_core_num"].str.replace(" nhân", "").astype(float)
df["cpu_thread_num"] = df["cpu_thread_num"].str.replace(" luồng", "").astype(float)

In [55]:
df.head()

,name,cpu,ram,monitor,gpu,memory,battery,connection,weight,price,...,connection_thunderbolt,ram_capacity,ram_type,ram_speed,HDD_capacity,SSD_capacity,cpu_brand,cpu_name,cpu_core_num,cpu_thread_num
0,Sạc Macbook Hyperjuice Ultimate Charger 110W D...,"Intel Core i5 1235U, 10 nhân, 12 luồng","8GB, DDR4, 3200 MHz","14"", 1920 x 1080 px, IPS, 65% sRGB, 60 Hz",Intel Iris Xe Graphics,SSD 512GB,39.3,"1 x Type-C, 1 x USB-A",1.400,3590000.0,...,0,8,DDR4,3200,0,512,Intel,Intel Core i5 1235U,10.0,12.0
1,Ghế Công Thái Học HyperWork HW01 (Arm 3D - Gray),"Intel Core i5 11300H, 4 nhân, 8 luồng","16GB, LPDDR4X, 4266 MHz","14.4"", 2400 x 1600 px, PixelSense Flow Display...",Intel Iris Xe Graphics,SSD 512GB,58.0,2 x Type-C,1.743,3790000.0,...,0,16,NaN,4266,0,512,Intel,Intel Core i5 11300H,4.0,8.0
2,Màn hình chuyên đồ hoạ ViewSonic VG2755-2K (QH...,"AMD Ryzen R5-5625U, 6 nhân, 12 luồng","8GB, DDR4, 3200 MHz","15.6"", 1920 x 1080 px, IPS, Chống chói, 65% sRGB",AMD Radeon Graphics,SSD 512GB,39.3,"1 x Type-C, 1 x USB-A",1.700,7790000.0,...,0,8,DDR4,3200,0,512,AMD,AMD Ryzen R5-5625U,6.0,12.0
3,Asus Vivobook X515 (Chính Hãng) (X515MA-BR481T),"Intel Celeron N4020, 2 nhân, 2 luồng","4GB, DDR4, 3200 MHz","15.6"", 1366 x 768 px, TN, Matte, 60 Hz",Intel UHD Graphics,"SSD 256GB, HDD 1024GB",37.0,"1 x Type-C, 3 x USB-A",NaN,7990000.0,...,0,4,DDR4,3200,1024,256,Intel,Intel Celeron N4020,2.0,2.0
4,ASUS Vivobook X515JA,"Intel Core i3 1005G1, 2 nhân, 4 luồng","8GB, 3200 MHz","15.6"", 1366 x 768 px, TN, Chống chói, 45% sRGB",Intel UHD Graphics,SSD 256GB,37.0,"1 x Type-C, 2 x USB-A",1.800,8990000.0,...,0,8,NaN,3200,0,256,Intel,Intel Core i3 1005G1,2.0,4.0


### Add one column: GPU brand name

In [56]:
with pd.option_context('display.max_rows', None,):
   print(df["gpu"].value_counts())

Intel Iris Xe Graphics                                      58
AMD Radeon Graphics                                         56
Intel Iris Xe                                               50
Intel UHD 620                                               27
Intel UHD Graphics                                          23
Intel UHD 630                                               23
Apple GPU                                                   16
Intel® Iris® Xe Graphics                                    16
Intel UHD                                                   12
Intel Iris XE                                                8
Radeon Graphics                                              8
Intel UHD G1                                                 6
Iris XE                                                      6
Iris Xe                                                      5
AMD Radeon™ Graphics                                         5
Iris Xe Graphics                                       

In [57]:
gpu_brand_dict = {"Intel":["Intel", "UHD"],
                  "AMD": ["Radeon", "AMD"],
                  "NVIDIA":["NVIDIA", "GeForce"]}

In [58]:
df["gpu"].isnull().sum()

3

In [59]:
def get_Intel_gpu(brand_name, gpu_brand_dict = gpu_brand_dict):
  for i in gpu_brand_dict["Intel"]:
    if i in brand_name:
      return 1
  return 0

def get_AMD_gpu(brand_name, gpu_brand_dict = gpu_brand_dict):
  for i in gpu_brand_dict["AMD"]:
    if i in brand_name:
      return 1
  return 0

def get_NVIDIA_gpu(brand_name, gpu_brand_dict = gpu_brand_dict):
  for i in gpu_brand_dict["NVIDIA"]:
    if i in brand_name:
      return 1
  return 0

In [60]:
df["Intel_gpu"] = df["gpu"].apply(lambda x: get_Intel_gpu(x) if pd.notnull(x) else x)
df["AMD_gpu"] = df["gpu"].apply(lambda x: get_AMD_gpu(x) if pd.notnull(x) else x)
df["NVIDIA_gpu"] = df["gpu"].apply(lambda x: get_NVIDIA_gpu(x) if pd.notnull(x) else x)

In [61]:
df.head()

,name,cpu,ram,monitor,gpu,memory,battery,connection,weight,price,...,ram_speed,HDD_capacity,SSD_capacity,cpu_brand,cpu_name,cpu_core_num,cpu_thread_num,Intel_gpu,AMD_gpu,NVIDIA_gpu
0,Sạc Macbook Hyperjuice Ultimate Charger 110W D...,"Intel Core i5 1235U, 10 nhân, 12 luồng","8GB, DDR4, 3200 MHz","14"", 1920 x 1080 px, IPS, 65% sRGB, 60 Hz",Intel Iris Xe Graphics,SSD 512GB,39.3,"1 x Type-C, 1 x USB-A",1.400,3590000.0,...,3200,0,512,Intel,Intel Core i5 1235U,10.0,12.0,1.0,0.0,0.0
1,Ghế Công Thái Học HyperWork HW01 (Arm 3D - Gray),"Intel Core i5 11300H, 4 nhân, 8 luồng","16GB, LPDDR4X, 4266 MHz","14.4"", 2400 x 1600 px, PixelSense Flow Display...",Intel Iris Xe Graphics,SSD 512GB,58.0,2 x Type-C,1.743,3790000.0,...,4266,0,512,Intel,Intel Core i5 11300H,4.0,8.0,1.0,0.0,0.0
2,Màn hình chuyên đồ hoạ ViewSonic VG2755-2K (QH...,"AMD Ryzen R5-5625U, 6 nhân, 12 luồng","8GB, DDR4, 3200 MHz","15.6"", 1920 x 1080 px, IPS, Chống chói, 65% sRGB",AMD Radeon Graphics,SSD 512GB,39.3,"1 x Type-C, 1 x USB-A",1.700,7790000.0,...,3200,0,512,AMD,AMD Ryzen R5-5625U,6.0,12.0,0.0,1.0,0.0
3,Asus Vivobook X515 (Chính Hãng) (X515MA-BR481T),"Intel Celeron N4020, 2 nhân, 2 luồng","4GB, DDR4, 3200 MHz","15.6"", 1366 x 768 px, TN, Matte, 60 Hz",Intel UHD Graphics,"SSD 256GB, HDD 1024GB",37.0,"1 x Type-C, 3 x USB-A",NaN,7990000.0,...,3200,1024,256,Intel,Intel Celeron N4020,2.0,2.0,1.0,0.0,0.0
4,ASUS Vivobook X515JA,"Intel Core i3 1005G1, 2 nhân, 4 luồng","8GB, 3200 MHz","15.6"", 1366 x 768 px, TN, Chống chói, 45% sRGB",Intel UHD Graphics,SSD 256GB,37.0,"1 x Type-C, 2 x USB-A",1.800,8990000.0,...,3200,0,256,Intel,Intel Core i3 1005G1,2.0,4.0,1.0,0.0,0.0


### Change the columns to the right data types

In [62]:
df.dtypes

name                          object
cpu                           object
ram                           object
monitor                       object
gpu                           object
memory                        object
battery                      float64
connection                    object
weight                       float64
price                        float64
brand                         object
monitor_size                  object
monitor_resolution_x          object
monitor_resolution_y          object
monitor_panel                 object
monitor_display               object
monitor_torch_screen           int64
monitor_blue_light_filter      int64
monitor_LED_backlit            int64
monitor_anti_glare             int64
monitor_sRGB                  object
monitor_refresh_rate          object
connection_type_c             object
connection_usb_a              object
connection_thunderbolt         int64
ram_capacity                  object
ram_type                      object
r

In [63]:
df["HDD_capacity"] = df["HDD_capacity"].astype(float)
df["SSD_capacity"] = df["SSD_capacity"].astype(float)
df["monitor_size"] = df["monitor_size"].astype(float)
df["monitor_resolution_x"] = df["monitor_resolution_x"].astype(float)
df["monitor_resolution_y"] = df["monitor_resolution_y"].astype(float)
df["monitor_sRGB"] = df["monitor_sRGB"].astype(float)
df["monitor_refresh_rate"] = df["monitor_refresh_rate"].astype(float)
df["connection_type_c"] = df["connection_type_c"].astype(float)
df["connection_usb_a"] = df["connection_usb_a"].astype(float)
df["ram_capacity"] = df["ram_capacity"].astype(float)
df["ram_speed"] = df["ram_speed"].astype(float)

In [64]:
df.dtypes

name                          object
cpu                           object
ram                           object
monitor                       object
gpu                           object
memory                        object
battery                      float64
connection                    object
weight                       float64
price                        float64
brand                         object
monitor_size                 float64
monitor_resolution_x         float64
monitor_resolution_y         float64
monitor_panel                 object
monitor_display               object
monitor_torch_screen           int64
monitor_blue_light_filter      int64
monitor_LED_backlit            int64
monitor_anti_glare             int64
monitor_sRGB                 float64
monitor_refresh_rate         float64
connection_type_c            float64
connection_usb_a             float64
connection_thunderbolt         int64
ram_capacity                 float64
ram_type                      object
r

In [65]:
df.drop(["ram","monitor","memory", "connection", "cpu"], axis = 1, inplace = True)
df.head()

,name,gpu,battery,weight,price,brand,monitor_size,monitor_resolution_x,monitor_resolution_y,monitor_panel,...,ram_speed,HDD_capacity,SSD_capacity,cpu_brand,cpu_name,cpu_core_num,cpu_thread_num,Intel_gpu,AMD_gpu,NVIDIA_gpu
0,Sạc Macbook Hyperjuice Ultimate Charger 110W D...,Intel Iris Xe Graphics,39.3,1.400,3590000.0,Other,14.0,1920.0,1080.0,IPS,...,3200.0,0.0,512.0,Intel,Intel Core i5 1235U,10.0,12.0,1.0,0.0,0.0
1,Ghế Công Thái Học HyperWork HW01 (Arm 3D - Gray),Intel Iris Xe Graphics,58.0,1.743,3790000.0,Other,14.4,2400.0,1600.0,None,...,4266.0,0.0,512.0,Intel,Intel Core i5 11300H,4.0,8.0,1.0,0.0,0.0
2,Màn hình chuyên đồ hoạ ViewSonic VG2755-2K (QH...,AMD Radeon Graphics,39.3,1.700,7790000.0,Other,15.6,1920.0,1080.0,IPS,...,3200.0,0.0,512.0,AMD,AMD Ryzen R5-5625U,6.0,12.0,0.0,1.0,0.0
3,Asus Vivobook X515 (Chính Hãng) (X515MA-BR481T),Intel UHD Graphics,37.0,NaN,7990000.0,Asus,15.6,1366.0,768.0,TN,...,3200.0,1024.0,256.0,Intel,Intel Celeron N4020,2.0,2.0,1.0,0.0,0.0
4,ASUS Vivobook X515JA,Intel UHD Graphics,37.0,1.800,8990000.0,Asus,15.6,1366.0,768.0,TN,...,3200.0,0.0,256.0,Intel,Intel Core i3 1005G1,2.0,4.0,1.0,0.0,0.0


In [66]:
df.dtypes

name                          object
gpu                           object
battery                      float64
weight                       float64
price                        float64
brand                         object
monitor_size                 float64
monitor_resolution_x         float64
monitor_resolution_y         float64
monitor_panel                 object
monitor_display               object
monitor_torch_screen           int64
monitor_blue_light_filter      int64
monitor_LED_backlit            int64
monitor_anti_glare             int64
monitor_sRGB                 float64
monitor_refresh_rate         float64
connection_type_c            float64
connection_usb_a             float64
connection_thunderbolt         int64
ram_capacity                 float64
ram_type                      object
ram_speed                    float64
HDD_capacity                 float64
SSD_capacity                 float64
cpu_brand                     object
cpu_name                      object
c

In [67]:
df.isnull().sum()

name                           0
gpu                            3
battery                       15
weight                        55
price                          0
brand                          0
monitor_size                   2
monitor_resolution_x          11
monitor_resolution_y          11
monitor_panel                 18
monitor_display              309
monitor_torch_screen           0
monitor_blue_light_filter      0
monitor_LED_backlit            0
monitor_anti_glare             0
monitor_sRGB                 264
monitor_refresh_rate          93
connection_type_c              0
connection_usb_a               0
connection_thunderbolt         0
ram_capacity                   2
ram_type                     132
ram_speed                     41
HDD_capacity                   0
SSD_capacity                   0
cpu_brand                      0
cpu_name                       0
cpu_core_num                   7
cpu_thread_num                17
Intel_gpu                      3
AMD_gpu   

In [68]:
cols = ['name',
        'brand',
        'cpu_name',
        'cpu_brand',
        'cpu_core_num',
        'cpu_thread_num',
        'gpu',
        'Intel_gpu',
        'AMD_gpu',
        'NVIDIA_gpu',
        'weight',
        'battery',
        'monitor_size',
        'monitor_resolution_x',
        'monitor_resolution_y',
        'monitor_panel',
        'monitor_display',
        'monitor_torch_screen',
        'monitor_blue_light_filter',
        'monitor_LED_backlit',
        'monitor_anti_glare',
        'monitor_sRGB',
        'monitor_refresh_rate',
        'connection_type_c',
        'connection_usb_a',
        'connection_thunderbolt',
        'ram_capacity',
        'ram_type',
        'ram_speed',
        'HDD_capacity',
        'SSD_capacity',
        'price']

In [69]:
df = df[cols]

In [70]:
df.shape

(560, 32)

In [71]:
df.head()

,name,brand,cpu_name,cpu_brand,cpu_core_num,cpu_thread_num,gpu,Intel_gpu,AMD_gpu,NVIDIA_gpu,...,monitor_refresh_rate,connection_type_c,connection_usb_a,connection_thunderbolt,ram_capacity,ram_type,ram_speed,HDD_capacity,SSD_capacity,price
0,Sạc Macbook Hyperjuice Ultimate Charger 110W D...,Other,Intel Core i5 1235U,Intel,10.0,12.0,Intel Iris Xe Graphics,1.0,0.0,0.0,...,60.0,1.0,1.0,0,8.0,DDR4,3200.0,0.0,512.0,3590000.0
1,Ghế Công Thái Học HyperWork HW01 (Arm 3D - Gray),Other,Intel Core i5 11300H,Intel,4.0,8.0,Intel Iris Xe Graphics,1.0,0.0,0.0,...,120.0,2.0,0.0,0,16.0,NaN,4266.0,0.0,512.0,3790000.0
2,Màn hình chuyên đồ hoạ ViewSonic VG2755-2K (QH...,Other,AMD Ryzen R5-5625U,AMD,6.0,12.0,AMD Radeon Graphics,0.0,1.0,0.0,...,NaN,1.0,1.0,0,8.0,DDR4,3200.0,0.0,512.0,7790000.0
3,Asus Vivobook X515 (Chính Hãng) (X515MA-BR481T),Asus,Intel Celeron N4020,Intel,2.0,2.0,Intel UHD Graphics,1.0,0.0,0.0,...,60.0,1.0,3.0,0,4.0,DDR4,3200.0,1024.0,256.0,7990000.0
4,ASUS Vivobook X515JA,Asus,Intel Core i3 1005G1,Intel,2.0,4.0,Intel UHD Graphics,1.0,0.0,0.0,...,NaN,1.0,2.0,0,8.0,NaN,3200.0,0.0,256.0,8990000.0


In [72]:
df.to_csv("/content/drive/MyDrive/DATA SCIENCE/Projects/Laptop price prediction/DS project/preprocessed_laptop_ver2.csv", index=False, encoding='utf-8-sig')

In [64]:
sheet_url = "https://docs.google.com/spreadsheets/d/1GtK-h_l3Cn-80MDOKWY2VgTr1OWRdAm_nfKOV2zrexw/edit#gid=0"
url_1 = sheet_url.replace('/edit#gid=', '/export?format=csv&gid=')

In [65]:
a.to_csv("/content/drive/MyDrive/DS project/laptop_schema.csv", index=False, encoding='utf-8-sig')

NameError: ignored

In [73]:
b = pd.read_csv("/content/drive/MyDrive/DATA SCIENCE/Projects/Laptop price prediction/DS project/preprocessed_laptop_ver2.csv")
b

,name,brand,cpu_name,cpu_brand,cpu_core_num,cpu_thread_num,gpu,Intel_gpu,AMD_gpu,NVIDIA_gpu,...,monitor_refresh_rate,connection_type_c,connection_usb_a,connection_thunderbolt,ram_capacity,ram_type,ram_speed,HDD_capacity,SSD_capacity,price
0,Sạc Macbook Hyperjuice Ultimate Charger 110W D...,Other,Intel Core i5 1235U,Intel,10.0,12.0,Intel Iris Xe Graphics,1.0,0.0,0.0,...,60.0,1.0,1.0,0,8.0,DDR4,3200.0,0.0,512.0,3590000.0
1,Ghế Công Thái Học HyperWork HW01 (Arm 3D - Gray),Other,Intel Core i5 11300H,Intel,4.0,8.0,Intel Iris Xe Graphics,1.0,0.0,0.0,...,120.0,2.0,0.0,0,16.0,NaN,4266.0,0.0,512.0,3790000.0
2,Màn hình chuyên đồ hoạ ViewSonic VG2755-2K (QH...,Other,AMD Ryzen R5-5625U,AMD,6.0,12.0,AMD Radeon Graphics,0.0,1.0,0.0,...,NaN,1.0,1.0,0,8.0,DDR4,3200.0,0.0,512.0,7790000.0
3,Asus Vivobook X515 (Chính Hãng) (X515MA-BR481T),Asus,Intel Celeron N4020,Intel,2.0,2.0,Intel UHD Graphics,1.0,0.0,0.0,...,60.0,1.0,3.0,0,4.0,DDR4,3200.0,1024.0,256.0,7990000.0
4,ASUS Vivobook X515JA,Asus,Intel Core i3 1005G1,Intel,2.0,4.0,Intel UHD Graphics,1.0,0.0,0.0,...,NaN,1.0,2.0,0,8.0,NaN,3200.0,0.0,256.0,8990000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
555,Razer Blade 15 Advanced (Intel Gen 11) (RZ09-0...,Razer,Intel Core i9 11900H,Intel,8.0,16.0,Intel UHD,1.0,0.0,0.0,...,60.0,2.0,3.0,1,32.0,DDR4,3200.0,0.0,1024.0,114990000.0
556,MSI Stealth GS77 2022,MSI,Intel Core i9-12900H,Intel,16.0,24.0,Nvidia Geforce RTX 3080Ti 16GB GDDR6,0.0,0.0,0.0,...,120.0,2.0,2.0,1,32.0,DDR5,4800.0,0.0,2048.0,119990000.0
557,Razer Blade 15 2022,Razer,Intel Core i9 12900H,Intel,14.0,20.0,"Intel® Iris® Xe Graphics eligible, RTX30080Ti",1.0,0.0,0.0,...,144.0,1.0,2.0,0,32.0,DDR5,4800.0,0.0,1024.0,129990000.0
558,Razer Blade 17 2022,Razer,Intel Core i9 12900H,Intel,14.0,20.0,"Intel Iris Xe Graphics, Nvidia Geforce RTX3080Ti",1.0,0.0,0.0,...,144.0,2.0,3.0,1,32.0,NaN,4800.0,0.0,1024.0,129990000.0
